In [1]:
%matplotlib inline

import os
# running with non gpu singularity container, so commented out the next line to use CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
tf.set_random_seed(42)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
print "import tensorflow"
           
import keras.backend.tensorflow_backend as K

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Lambda
from keras.layers import Input, merge, Concatenate, concatenate, Add, Multiply
from keras.losses import binary_crossentropy
from keras.optimizers import RMSprop,Adadelta
print "import keras"

import numpy as np
# from tqdm import tqdm
import time
import pickle
import sys
import glob

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

print "import matplotlib"

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import binned_statistic_2d
from scipy.stats import ks_2samp


print "import sklearn"

np.random.seed(42)


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


import tensorflow
import keras
import matplotlib


Using TensorFlow backend.


import sklearn


In [2]:
def Minv(cols,ptetaphi=False,nopy2=True):
    """
    Computes M for two objects given the cartesian momentum projections
    if `ptetaphi` is True, then assumes the 8 input columns are cylindrical eptetaphi
    if `nopy2` is True, input is 7 columns with no py2
    """
    if ptetaphi:
        cols = ptetaphi_to_cartesian(cols)
    if nopy2:
        M2 = (cols[:,0]+cols[:,4])**2
        M2 -= (cols[:,1]+cols[:,5])**2
        M2 -= (cols[:,2]          )**2
        M2 -= (cols[:,3]+cols[:,6])**2
    else:
        M2 = (cols[:,0]+cols[:,4])**2
        M2 -= (cols[:,1]+cols[:,5])**2
        M2 -= (cols[:,2]+cols[:,6])**2
        M2 -= (cols[:,3]+cols[:,7])**2
    return np.sqrt(M2)

def cartesian_to_ptetaphi(eight_cartesian_cols):
    """
    Takes 8 columns as cartesian e px py pz e px py pz
    and converts to e pt eta phi e pt eta phi
    """
    e1 =  eight_cartesian_cols[:,0]
    e2 =  eight_cartesian_cols[:,4]
    px1 = eight_cartesian_cols[:,1]
    px2 = eight_cartesian_cols[:,5]
    py1 = eight_cartesian_cols[:,2]
    py2 = eight_cartesian_cols[:,6]
    pz1 = eight_cartesian_cols[:,3]
    pz2 = eight_cartesian_cols[:,7]
    p1 = np.sqrt(px1**2+py1**2+pz1**2)
    p2 = np.sqrt(px2**2+py2**2+pz2**2)
    pt1 = np.sqrt(px1**2+py1**2)
    pt2 = np.sqrt(px2**2+py2**2)
    phi1 = np.arctan2(py1,px1)
    phi2 = np.arctan2(py2,px2)
    eta1 = np.arctanh(pz1/p1)
    eta2 = np.arctanh(pz2/p2)
    return np.c_[e1,pt1,eta1,phi1,e2,pt2,eta2,phi2]

def ptetaphi_to_cartesian(eight_eptetaphi_cols):
    """
    Takes 8 columns as e pt eta phi e pt eta phi
    and converts to e px py pz e px py pz
    """
    e1 =  eight_eptetaphi_cols[:,0]
    e2 =  eight_eptetaphi_cols[:,4]
    pt1 =  eight_eptetaphi_cols[:,1]
    pt2 =  eight_eptetaphi_cols[:,5]
    eta1 =  eight_eptetaphi_cols[:,2]
    eta2 =  eight_eptetaphi_cols[:,6]
    phi1 =  eight_eptetaphi_cols[:,3]
    phi2 =  eight_eptetaphi_cols[:,7]
    px1 = np.abs(pt1)*np.cos(phi1)
    px2 = np.abs(pt2)*np.cos(phi2)
    py1 = np.abs(pt1)*np.sin(phi1)
    py2 = np.abs(pt2)*np.sin(phi2)
    pz1 = np.abs(pt1)/np.tan(2.0*np.arctan(np.exp(-1.*eta1)))
    pz2 = np.abs(pt2)/np.tan(2.0*np.arctan(np.exp(-1.*eta2)))
    return np.c_[e1,px1,py1,pz1,e2,px2,py2,pz2]

def get_dphi(px1,py1,px2,py2):
    phi1 = np.arctan2(py1,px1)
    phi2 = np.arctan2(py2,px2)
    dphi = phi1-phi2
    dphi[dphi>np.pi] -= 2*np.pi
    dphi[dphi<-np.pi] += 2*np.pi 
    return dphi

def M4(E,px,py,pz):
    return np.sqrt(E*E - px*px - py*py - pz*pz)


In [3]:
def invmass_from_8cartesian_nopy2(x):
    
    invmass = K.sqrt(
                (x[:,0:1]+x[:,4:5])**2-
                (x[:,1:2]+x[:,5:6])**2-
                (x[:,2:3]         )**2-
                (x[:,3:4]+x[:,6:7])**2
                )
    return invmass


# def getKS(real_data, predictions):
#     return ks_2samp(real_data["mll"], Minv(predictions))

def getKS(real_data, predictions):
    return ks_2samp(real_data["mll"], Minv(predictions))[0] + \
     ks_2samp(real_data["lep1_e"], predictions[:,0])[0] + \
     ks_2samp(real_data["lep1_px"], predictions[:,1])[0] + \
     ks_2samp(real_data["lep1_py"], predictions[:,2])[0] + \
     ks_2samp(real_data["lep1_pz"], predictions[:,3])[0] + \
     ks_2samp(real_data["lep1_e"], predictions[:,4])[0] + \
     ks_2samp(real_data["lep2_px"], predictions[:,5])[0] + \
     ks_2samp(real_data["lep2_pz"], predictions[:,6])[0] + \
     ks_2samp(real_data["nvtxs"], np.rint(predictions[:,7]))[0] + \
     ks_2samp(real_data["lep1_iso"], predictions[:,8])[0] + \
     ks_2samp(real_data["lep2_iso"], predictions[:,9])[0] + \
     ks_2samp(real_data["met"]*np.cos(real_data["metphi"]), predictions[:,10])[0] + \
     ks_2samp(real_data["met"]*np.sin(real_data["metphi"]), predictions[:,11])[0] + \
     ks_2samp(real_data["jet_pt1"], predictions[:,12])[0] + \
     ks_2samp(real_data["jet_pt2"], predictions[:,13])[0] + \
     ks_2samp(real_data["jet_pt3"], predictions[:,14])[0] + \
     ks_2samp(real_data["jet_pt4"], predictions[:,15])[0] + \
     ks_2samp(real_data["jet_pt5"], predictions[:,16])[0]

# def get_first_N(x,N):
#     return x[:,0:N]

# def fix_outputs(x):
#     """
#     Take nominal delphes format of 19 columns and fix some columns
#     """
#     return K.concatenate([
#         # x[:,0:21],
#         x[:,0:7], # epxpypz for lep1,lep2 -1 for no py2
#         x[:,7:8], # nvtx
#         K.sign(x[:,8:10]), # q1 q2
#         x[:,10:12], # iso1 iso2
#         x[:,12:14], # met, metphi
#         x[:,14:19], # jet pts
#         ])



In [4]:
def make_plots(preds,reals,title="",fname="",show_pred=True,show_real=True,wspace=0.1,hspace=0.3,tightlayout=True,visible=True):
    nrows, ncols = 5,5
    fig, axs = plt.subplots(nrows,ncols,figsize=(16,13))
#     fig, axs = plt.subplots(nrows,ncols,figsize=(12,10))
#     fig.subplots_adjust(wspace=0.1,hspace=0.3)
    fig.subplots_adjust(wspace=wspace,hspace=hspace)


    info = [
        ["lep1_e",(0,250,50)],
        ["lep1_px",(-100,100,50)],
        ["lep1_py",(-100,100,50)],
        ["lep1_pz",(-200,200,50)],
        ["lep2_e",(0,250,50)],
        ["lep2_px",(-100,100,50)],
        ["lep2_pz",(-200,200,50)],
        ["nvtxs",(0,50,350)],
        ["metx",(-50,50,100)],
        ["mety",(-50,50,100)],
#         ["lep1_charge",(-7,7,30)],
#         ["lep2_charge",(-7,7,30)],
        ["lep1_iso",(0,2.0,30)],
        ["lep2_iso",(0,2.0,30)],
        ["jet_pt1",(0,100,50)],
        ["jet_pt2",(0,100,50)],
        ["jet_pt3",(0,100,50)],
        ["jet_pt4",(0,100,50)],
        ["jet_pt5",(0,100,50)],
        # derived features
        ["dphi",(-4,4,50)],
        ["met",(0,150,50)],
        ["metphi",(-6,6,50)],
        ["mll",(60,120,50)],
        ["lep1_mass",(0,10,50)],
        ["lep2_mass",(0,10,50)],
        ["njets",(0,7,7)],
    ]
    for axx in axs:
        for ax in axx:
            ax.get_yaxis().set_visible(False)
            # turn off all axis borders, and turn them on below so they only show
            # up for axes we've plotted in
            ax.axis('off')
    for ic,(cname,crange) in enumerate(info):
        if cname == "mll":
            real = reals["mll"]
            pred = Minv(preds)
        elif cname == "lep1_mass": real, pred = M4(reals["lep1_e"], reals["lep1_px"], reals["lep1_py"], reals["lep1_pz"]), M4(preds[:,0], preds[:,1], preds[:,2], preds[:,3])
        elif cname == "lep2_mass": real, pred = M4(reals["lep2_e"], reals["lep2_px"], 0, reals["lep2_pz"]), M4(preds[:,4], preds[:,5], preds[:,6], preds[:,7])
        elif cname == "lep1_e": real, pred = reals[cname], preds[:,0]
        elif cname == "lep1_pz": real, pred = reals[cname], preds[:,3]
        elif cname == "lep2_e": real, pred = reals[cname], preds[:,4]
        elif cname == "lep2_pz": real, pred = reals[cname], preds[:,6]
        elif cname == "lep1_px": 
            real = reals[cname]
            pred = preds[:,1]
        elif cname == "lep1_py":
            real = reals[cname]
            pred = preds[:,2]
        elif cname == "lep2_px":
            real = reals[cname]
            pred = preds[:,5]
        elif cname == "dphi":
            real = get_dphi(reals["lep1_px"], reals["lep1_py"], reals["lep2_px"], np.zeros(len(reals)))
            pred = get_dphi(preds[:,1], preds[:,2], preds[:,5], np.zeros(len(preds)))
        elif cname == "nvtxs": real, pred = reals[cname], np.round(preds[:,7])
#         elif cname == "lep1_charge": real, pred = reals[cname], preds[:,8]
#         elif cname == "lep2_charge": real, pred = reals[cname], preds[:,9]
        elif cname == "lep1_iso": real, pred = reals[cname], preds[:,8]
        elif cname == "lep2_iso": real, pred = reals[cname], preds[:,9]
        elif cname == "metx": real, pred = reals["met"]*np.cos(reals["metphi"]), preds[:,10]
        elif cname == "mety": real, pred = reals["met"]*np.sin(reals["metphi"]), preds[:,11]
        elif cname == "met": real, pred = reals["met"], np.hypot(preds[:,10],preds[:,11])
        elif cname == "metphi": real, pred = reals["metphi"], np.arctan2(preds[:,11],preds[:,10])
        elif cname == "jet_pt1": real, pred = reals[cname], preds[:,12]
        elif cname == "jet_pt2": real, pred = reals[cname], preds[:,13]
        elif cname == "jet_pt3": real, pred = reals[cname], preds[:,14]
        elif cname == "jet_pt4": real, pred = reals[cname], preds[:,15]
        elif cname == "jet_pt5": real, pred = reals[cname], preds[:,16]
        elif cname == "njets":
            real = \
                1*(reals["jet_pt1"] > 15) + \
                1*(reals["jet_pt2"] > 15) + \
                1*(reals["jet_pt3"] > 15) + \
                1*(reals["jet_pt4"] > 15) + \
                1*(reals["jet_pt5"] > 15)
            pred = \
                1*(preds[:,12] > 15) + \
                1*(preds[:,13] > 15) + \
                1*(preds[:,14] > 15) + \
                1*(preds[:,15] > 15) + \
                1*(preds[:,16] > 15)
        idx = ic // ncols, ic % ncols
        if show_real:
            bins_real = axs[idx].hist(real, range=crange[:2],bins=crange[-1], histtype="step", lw=1.5,density=True)
        if show_pred:
            bins_pred = axs[idx].hist(pred, range=crange[:2],bins=crange[-1], histtype="step", lw=1.5,density=True)
        axs[idx].set_xlabel("{}".format(cname),fontsize=14)
        axs[idx].axis('on')
        if cname in ["mll","lep1_mass","lep2_mass","dphi","met","metphi","njets"]:
            axs[idx].xaxis.label.set_color('blue')
    #     axs[idx].set_yscale("log", nonposy='clip')
    _ = axs[0,0].legend(["True","Pred"], loc='upper right',fontsize=14)
    _ = axs[0,0].set_title(title)
    if tightlayout:
        plt.tight_layout()
    if fname:
        fig.savefig(fname)
    if not visible:
        plt.close(fig)
        

In [5]:
def get_recview(data, dtype="<f4"):
    """
    for example, can give it a matrix of values predicted by generator
    and this gives a view of the same matrix with dtypes (for easier column selection)
    if weird results, like 2 columns per field, use dtype="<f8"
    """
    cnames = [
    "lep1_e",
    "lep1_px",
    "lep1_py",
    "lep1_pz",
    "lep2_e",
    "lep2_px",
    "lep2_pz",
    "nvtxs",
#     "lep1_charge",
#     "lep2_charge",
    "lep1_iso",
    "lep2_iso",
    "metx",
    "mety",
    "jet_pt1",
    "jet_pt2",
    "jet_pt3",
    "jet_pt4",
    "jet_pt5",
    ]
    cnames = [(cn,dtype) for cn in cnames]
    return data.view(dtype=cnames)

## GAN class
Instantiate the GAN class, then add methods piece by piece (by making a "new" class inheriting from the original).
I did this so that all these long functions could go into different cells, which makes it easier to navigate/read.

In [6]:
class GAN():
    def __init__(self, **kwargs):

        self.args = dict(kwargs)

        self.verbose = kwargs.get("verbose",True)
        self.tag = kwargs["tag"]
        self.input_file = str(kwargs["input_file"])
        self.noise_shape = (int(kwargs["noise_size"]),)
        self.output_shape = (int(kwargs["output_size"]),)
        self.noise_type = int(kwargs["noise_type"])
        self.ntest_samples = int(kwargs["ntest_samples"])
        self.nepochs_dump_pred_metrics = int(kwargs["nepochs_dump_pred_metrics"])
        self.nepochs_dump_models = int(kwargs["nepochs_dump_models"])
        self.nepochs_dump_plots = int(kwargs["nepochs_dump_plots"])
        self.nepochs_max = int(kwargs["nepochs_max"])
        self.batch_size = int(kwargs["batch_size"])
        self.do_soft_labels = kwargs["do_soft_labels"]
        self.do_noisy_labels = kwargs["do_noisy_labels"]
        self.nepochs_decay_noisy_labels = int(kwargs["nepochs_decay_noisy_labels"])
        self.optimizer_gen = kwargs["optimizer_gen"]
        self.optimizer_disc = kwargs["optimizer_disc"]
        self.depth_disc = kwargs["depth_disc"]
        self.width_disc = kwargs["width_disc"]
        self.depth_gen = kwargs["depth_gen"]
        self.width_gen = kwargs["width_gen"]
        self.beefy_generator = kwargs["beefy_generator"]
        self.beefy_discriminator = kwargs["beefy_discriminator"]
#         self.fix_delphes_outputs = kwargs["fix_delphes_outputs"]
        self.use_mll_loss = kwargs["use_mll_loss"]
        self.loss_mll_weight = kwargs["loss_mll_weight"]
        self.terminate_early = kwargs["terminate_early"]
        self.loss_type = kwargs["loss_type"]
        self.dropout_discriminator = kwargs["dropout_discriminator"]
        self.frac_true = kwargs["frac_true"]

        os.system("mkdir -p progress/{}/".format(self.tag))
        os.system("cp gan_reco.ipynb progress/{}/".format(self.tag))

        self.scaler_type = kwargs["scaler_type"]
        self.scaler = None
        self.jetscaler = None
        self.isoscaler = None
        if self.scaler_type.lower() == "minmax":
            self.scaler = MinMaxScaler(feature_range=(-1.,1.))
        elif self.scaler_type.lower() == "robust":
            self.scaler = RobustScaler()
        elif self.scaler_type.lower() == "standard":
            self.scaler = StandardScaler()
        else:
            if "jet" in self.scaler_type.lower():
                print "Scaling jet pts"
                self.jetscaler = MinMaxScaler(feature_range=(-1.,1.))
            if "iso" in self.scaler_type.lower():
                print "Scaling lep isos"
                self.isoscaler = MinMaxScaler(feature_range=(-1.,1.))

        self.data = None
        self.data_ref = None
        self.d_epochinfo = {}
        self.X_train = None

        optimizer_d = self.optimizer_disc
        optimizer_g = self.optimizer_gen
        
        def dosplit(x):
            return float(x.split("=")[1].split(")")[0].strip())
        if "lr" in self.optimizer_disc: optimizer_d = Adadelta(lr=dosplit(self.optimizer_disc))
        if "lr" in self.optimizer_gen: optimizer_g = Adadelta(lr=dosplit(self.optimizer_gen))

        # Build and compile the discriminator
        if self.use_mll_loss:
            loss = self.custom_loss(c=self.loss_mll_weight, loss_type=self.loss_type)
        else:
            loss = "binary_crossentropy"
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=loss,
            optimizer=optimizer_d,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss=loss, optimizer=optimizer_g)

        # The generator takes noise as input and generated imgs
        z = Input(shape=self.noise_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss=loss, optimizer=optimizer_g)


In [7]:
class GAN(GAN):
    
    def custom_loss(self, c, loss_type = "force_mll"):
        mu_z, sig_z = 89.6, 7.73
        if loss_type == "disc":
            def loss_func(y_true, y_pred_mll):
                y_true = y_true[:,0]
                y_pred = y_pred_mll[:,0]
                return binary_crossentropy(y_true, y_pred)
            return loss_func
        elif loss_type == "force_mll":
            def loss_func(y_true, y_pred_mll):
                y_true = y_true[:,0]
                y_pred = y_pred_mll[:,0]
                mll_pred = y_pred_mll[:,1]
                mll_loss = K.mean(K.abs(mll_pred - mu_z))
                return binary_crossentropy(y_true, y_pred) + c*mll_loss
            return loss_func
        elif loss_type == "force_z_width":
            def loss_func(y_true, y_pred_mll):
                y_true = y_true[:,0]
                y_pred = y_pred_mll[:,0]
                mll_pred = y_pred_mll[:,1]
                mll_loss = K.mean((mll_pred - mu_z)**2)
                mll_sigma_loss = (K.std(mll_pred)-sig_z)**2

                return binary_crossentropy(y_true, y_pred) + c*mll_loss + c*mll_sigma_loss
            return loss_func
        else:
            raise ValueError("Can not make loss function of type %s" % loss_type)
        
    def build_generator(self):

        inputs = Input(shape=self.noise_shape)

        ## Head
        x = Dense(64)(inputs)
        x = LeakyReLU(alpha=0.2)(x)
        if self.depth_gen > 0 and self.width_gen > 0:
            for level in xrange(0,self.depth_gen):
                x = Dense(width_gen/(2**level))(x) #Triangle with width halved at each level
                x = LeakyReLU(alpha=0.2)(x)
        elif self.beefy_generator:
            for size in [128,256,512,256,128]:
                x = Dense(size)(x)
                x = LeakyReLU(alpha=0.2)(x)
        else:
            for size in [128,128,128,64,32]:
                x = Dense(size)(x)
                x = LeakyReLU(alpha=0.2)(x)
 
    
        x = Dense(self.output_shape[0], activation="linear")(x)
            
#         if self.fix_delphes_outputs:
#             x = Lambda(fix_outputs,
#                 input_shape=self.output_shape,
#                 output_shape=self.output_shape
#                 )(x)
            
        model = Model(inputs=inputs, outputs=[x])
        
        print "Generator params: {}".format(model.count_params())
        if self.verbose:
            model.summary()
        
        return model
    
class GAN(GAN):
    
    def build_discriminator(self):

        inputs = Input(self.output_shape)
        mll = Lambda(invmass_from_8cartesian_nopy2)(inputs)
        x = Dense(128)(inputs)
        x = LeakyReLU(alpha=0.2)(x)

        ## Main Body
        if self.depth_disc > 0 and self.width_disc > 0:
            for level in xrange(0,self.depth_disc):
                x = Dense(self.width_disc/(2**level))(x) #Triangle with width halved at each level
                x = LeakyReLU(alpha=0.2)(x)
        elif self.beefy_discriminator:
            for size in [128,256,256,128,64,32,16,8]:
                x = Dense(size)(x)
                if self.dropout_discriminator:
                    x = Dropout(0.1)(x)
                x = LeakyReLU(alpha=0.2)(x)
        else:
            for size in [128]*5 + [64,32,16,8]:
                x = Dense(size)(x)
                x = LeakyReLU(alpha=0.2)(x)

        ## Tail
        out = Dense(1,activation='sigmoid')(x)
        
        if self.use_mll_loss:
            model = Model(inputs=inputs, outputs=concatenate([out,mll]))
        else:
            model = Model(inputs=inputs, outputs=out)
#         print model.output_shape
        if self.verbose:
            model.summary()
        print "Discriminator params: {}".format(model.count_params())
        
        return model
        

In [8]:
class GAN(GAN):
    
    def load_data(self):
        if self.data is not None: return
        
        self.data = np.load(self.input_file)
            
        # make sure we drop low mass resonances
        self.data = self.data[self.data["genmll"] > 50.]
        
    def make_flat_array(self):
        """Builds X_train array which is a flat version of the self.data that has any scaling or modifications applied"""
        
        lepcoords = np.c_[
            self.data["lep1_e"],
            self.data["lep1_px"],
            self.data["lep1_py"],
            self.data["lep1_pz"],
            self.data["lep2_e"],
            self.data["lep2_px"],
            self.data["lep2_pz"],
        ]

        nvtx_smeared = np.round(np.random.normal(self.data["nvtxs"],0.5))
        
        isocoords = np.c_[self.data["lep1_iso"], 
                          self.data["lep2_iso"]]
        if self.isoscaler:
            print "scaling lepton isolations"
            self.isoscaler.fit(isocoords)
            isocoords = self.isoscaler.transform(isocoords).astype(np.float32)
            pickle.dump(self.isoscaler, open("progress/{}/isoscaler.pkl".format(self.tag),'w'))
        
        jetcoords = np.c_[self.data["jet_pt1"],
                          self.data["jet_pt2"],
                          self.data["jet_pt3"],
                          self.data["jet_pt4"],
                          self.data["jet_pt5"]]
        
        if self.jetscaler:
            print "scaling jet pts"
            self.jetscaler.fit(jetcoords)
            jetcoords = self.jetscaler.transform(jetcoords).astype(np.float32)
            pickle.dump(self.jetscaler, open("progress/{}/jetscaler.pkl".format(self.tag),'w'))
        
        self.X_train = np.c_[
            lepcoords, # 7 columns
            nvtx_smeared, # 1 column
            isocoords,
            self.data["met"]*np.cos(self.data["metphi"]), # metx
            self.data["met"]*np.sin(self.data["metphi"]), # mety
            jetcoords
        ].astype(np.float32)

        # # NOTE. StandardScaler should be fit on training set
        # # and applied the same to train and test, otherwise we
        # # introduce a bias
        if self.scaler:
            self.scaler.fit(self.X_train)
            self.X_train = self.scaler.transform(self.X_train).astype(np.float32)
            pickle.dump(self.scaler, open("progress/{}/scaler.pkl".format(self.tag),'w'))
        

    def get_noise(self, amount=1024, max_true_samples=-1, max_true_samples_frac=-1):
        """
        `amount` specifies number of noise vectors
        `max_true_samples` applies only to truth conditioned noise type
            if > 0, then the true samples are sampled from the first 
            `max_true_samples` of real events (by default, all are allowed)
        `max_true_samples_frac` same desc as `max_true_samples`, but specified
            instead as fraction of `amount`
        """
        # nominal
        if self.noise_type == 1:
            noise_half = np.random.normal(0, 1, (amount//2, self.noise_shape[0]))
            noise_full = np.random.normal(0, 1, (amount, self.noise_shape[0]))

        elif self.noise_type == 2: # random soup, 4,2,2 have to be modified to sum to noise_shape[0]
            ngaus = self.noise_shape[0] // 2
            nflat = (self.noise_shape[0] - ngaus) // 2
            nexpo = self.noise_shape[0] - nflat - ngaus
            noise_gaus = np.random.normal( 0, 1, (amount//2+amount, ngaus))
            noise_flat = np.random.uniform(-1, 1, (amount//2+amount, nflat))
            noise_expo = np.random.exponential( 1,    (amount//2+amount, nexpo))
            noise = np.c_[ noise_gaus,noise_flat,noise_expo ]
            noise_half = noise[:amount//2]
            noise_full = noise[-amount:]
        elif self.noise_type == 3: #Flat noise between 0-1, last 4 units are flipped negative
            noise_half = np.random.uniform(0, 1, (amount//2, self.noise_shape[0]))
            noise_half[:,-4:] *= -1
            noise_full = np.random.uniform(0, 1, (amount, self.noise_shape[0]))
            noise_full[:,-4:] *= -1
            
            
        return noise_half, noise_full
    
    def get_available_checkpoints(self):
        # check the output folder for all weights files and return list of epochs for existing files
        fnames = glob.glob("progress/{}/gen_*.weights".format(self.tag))
        return np.array(sorted(map(lambda x: int(x.rsplit("_",1)[1].split(".")[0]), fnames)))

    def load_checkpoint(self, epoch):
        # given an epoch number, load the disc/gen files, overriding self.discriminator/self.generator
        # need to give custom_objects to load_model because keras doesn't know what the loss function is otherwise
        custom = {"loss_func": gan.custom_loss(c=self.loss_mll_weight,loss_type=self.loss_type)}
        # make sure data is loaded if we just want to make a gan, load a checkpoint, and predict + pull real samples
        self.load_data()
        self.discriminator = load_model("progress/{}/disc_{}.weights".format(self.tag,epoch),custom_objects=custom)
        self.generator = load_model("progress/{}/gen_{}.weights".format(self.tag,epoch),custom_objects=custom)
        self.d_epochinfo = pickle.load(open("progress/{}/history.pkl".format(self.tag),'r'))

        
    def load_last_checkpoint(self,which=-1):
        # convenience function to get last available checkpoint and load it (or `which`th from last)
        lastepoch = self.get_available_checkpoints()[which]
        print "Loading last checkpoint for tag {}: epoch {}".format(self.tag, lastepoch)
        self.load_checkpoint(lastepoch)
        
    def predict(self, N, frac):
        """Gets prediction from trained generator and undoes any scaling."""
        _, noise = self.get_noise(N,max_true_samples_frac=frac)
        # print preds.shape
        preds = self.generator.predict(noise,verbose=1)
        
        if self.scaler:
            preds = self.scaler.inverse_transform(preds)
            return preds
        
        lepcoords = preds[:,0:7]
        nvtx = preds[:,7]
        isocoords = preds[:,8:10]
        metcoords = preds[:,10:12]
        jetcoords = preds[:,12:17]
        if self.isoscaler:
            isocoords = self.isoscaler.inverse_transform(isocoords)
        if self.jetscaler:
            jetcoords = self.jetscaler.inverse_transform(jetcoords)
            
        return  np.c_[
                lepcoords, # 7 columns
                nvtx, # 1 column
                isocoords,
                metcoords,
                jetcoords
            ].astype(np.float32)
        




In [9]:
class GAN(GAN):
            
    def train(self):

        self.load_data()
        self.make_flat_array()
        
        # make an alias to save typing
        X_train = self.X_train
        
        half_batch = int(self.batch_size / 2)

        prev_gen_loss = -1
        prev_disc_loss = -1
        n_loss_same_gen = 0  # number of epochs for which generator loss has remained ~same (within 0.01%)
        n_loss_same_disc = 0  # number of epochs for which discriminator loss has remained ~same (within 0.01%)
        old_info = -1, -1
        ks_score = 999.
        best_ks_score = 999.
        for epoch in range(self.nepochs_max):

            if self.terminate_early:
                if n_loss_same_gen > 1000 or n_loss_same_disc > 1000:
                    print "BREAKING because disc/gen loss has remained the same for {}/{} epochs!".format(n_loss_same_disc,n_loss_same_gen)
                    break

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]
            
            noise_half, noise_full = self.get_noise(self.batch_size, max_true_samples_frac=self.frac_true)
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise_half)

            # Train the discriminator
            ones = np.ones((half_batch, 1))
            zeros = np.zeros((half_batch, 1))

            if self.do_soft_labels:
                ones *= 0.9

            if self.do_noisy_labels:
                frac = 0.3*np.exp(-epoch/self.nepochs_decay_noisy_labels)
                if frac > 0.005:
                    ones[np.random.randint(0, len(ones), int(frac*len(ones)))] = 0
                    zeros[np.random.randint(0, len(zeros), int(frac*len(zeros)))] = 1

            d_loss_real = self.discriminator.train_on_batch(imgs, ones)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, zeros)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # ---------------------
            #  Train Generator
            # ---------------------
            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * self.batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise_full, valid_y)

            if (g_loss - prev_gen_loss) < 0.0001: n_loss_same_gen += 1
            else: n_loss_same_gen = 0
            prev_gen_loss = g_loss

            if (d_loss[0] - prev_disc_loss) < 0.0001: n_loss_same_disc += 1
            else: n_loss_same_disc = 0
            prev_disc_loss = d_loss[0]

            # Plot the progress
#             print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            sys.stdout.write("\r{} [D loss: {}, acc.: {:.2f}%] [G loss: {}] [mll={:.3f}+-{:.3f}] [ks={:.3f}]".format(epoch, d_loss[0], 100.0*d_loss[1], g_loss, old_info[0], old_info[1],ks_score))

            if epoch % self.nepochs_dump_pred_metrics == 0 and epoch > 0:
            
                _, noise_test = self.get_noise(self.ntest_samples,  max_true_samples_frac=self.frac_true)
            
                sys.stdout.write("\n") # break up the stream of text

                gen_imgs = self.generator.predict(noise_test)

                if self.scaler:
                    gen_imgs = self.scaler.inverse_transform(gen_imgs)

                masses = Minv(gen_imgs)
                masses = masses[np.isfinite(masses)]
                old_info = masses.mean(), masses.std()
                ks_score = getKS(self.data[:15000],gen_imgs)

                if "epoch" not in self.d_epochinfo:
                    self.d_epochinfo["epoch"] = []
                    self.d_epochinfo["d_acc"] = []
                    self.d_epochinfo["d_loss"] = []
                    self.d_epochinfo["g_loss"] = []
                    self.d_epochinfo["mass_mu"] = []
                    self.d_epochinfo["mass_sig"] = []
                    self.d_epochinfo["ks"] = []
                    self.d_epochinfo["time"] = []
                    self.d_epochinfo["args"] = self.args
                else:
                    self.d_epochinfo["epoch"].append(epoch)
                    self.d_epochinfo["d_acc"].append(100*d_loss[1])
                    self.d_epochinfo["d_loss"].append(d_loss[0])
                    self.d_epochinfo["g_loss"].append(g_loss)
                    self.d_epochinfo["mass_mu"].append(masses.mean())
                    self.d_epochinfo["mass_sig"].append(masses.std())
                    self.d_epochinfo["ks"].append(ks_score)
                    self.d_epochinfo["time"].append(time.time())

                pickle.dump(self.d_epochinfo, open("progress/{}/history.pkl".format(self.tag),'w'))

                # note, nested within nepochs_dump_pred_metrics, so below value must be multiple of that
                if epoch % self.nepochs_dump_plots == 0 and epoch > 0:
                    preds = self.predict(self.ntest_samples,self.frac_true)
                    reals = self.data[:15000]
                    _ = make_plots(preds,reals,title="{}: epoch {}".format(self.tag,epoch),
                                   fname="progress/{}/plots_{:06d}.png".format(self.tag,epoch),visible=False)

                # note, nested within nepochs_dump_pred_metrics, so below value must be multiple of that
                if epoch % self.nepochs_dump_models == 0 and epoch > 0:                    
                    dfname = "progress/{}/disc_{}.weights".format(self.tag,epoch)
                    gfname = "progress/{}/gen_{}.weights".format(self.tag,epoch)
                    self.discriminator.save(dfname)
                    self.generator.save(gfname)
                    if (ks_score < best_ks_score):
                        print "KS score improved from {:.2f} to {:.2f}, saving models to {}".format(best_ks_score,ks_score,gfname)
                        best_ks_score = ks_score

## Set parameters and instantiate
Parameters, plots, epoch metrics, get saved to folder `progress/<tag>`.

In [10]:
# defaults
params = {
        "batch_size": 512,
        "beefy_discriminator": True,
        "beefy_generator": True,
        "depth_disc": 0,
        "depth_gen": 0,
        "do_batch_normalization_disc": False,
        "do_batch_normalization_gen": False,
        "do_concatenate_disc": False,
        "do_concatenate_gen": False,
        "do_noisy_labels": False,
        "do_soft_labels": False,
#         "do_noisy_labels": True,
#         "do_soft_labels": True,
        "do_skip_connection": False,
        "do_tanh_gen": False,
        "dropout_discriminator": False,
        "frac_true": -1,
        "input_file": "/home/users/bhashemi/Projects/GIT/DY-GAN/delphes/total_Zmumu_13TeV_PU20_v2.npa",
        "loss_mll_weight": 0.0001,
#         "loss_type": "force_mll",
        "loss_type": "force_z_width",
        "nepochs_decay_noisy_labels": 1000,
#         "nepochs_dump_models": 9999999, # can't dump models? FIXME
        "nepochs_dump_models": 100, # can't dump models? FIXME
        "nepochs_dump_plots": 500,
        "nepochs_dump_pred_metrics": 100,
        "nepochs_max": 100001,
        "noise_size": 17,
        "noise_type": 3,
        "ntest_samples": 10000,
        "optimizer_disc": "adadelta",
        "optimizer_gen": "adadelta",
#         "optimizer_disc": "adadelta(lr=0.1)",
#         "optimizer_gen": "adadelta(lr=0.1)",
        "output_size": 17,
        "scaler_type": "jetiso",
        "terminate_early": True,
        "use_mll_loss": True,
        "width_disc": 0,
        "width_gen": 0,
        "verbose": True,
        }
params.update({
})
print params
    
# change tag for provenance
params["tag"] = "v3_test"

gan = GAN(**params)


{'do_skip_connection': False, 'width_disc': 0, 'ntest_samples': 10000, 'optimizer_disc': 'adadelta', 'nepochs_dump_models': 100, 'loss_type': 'force_z_width', 'input_file': '/home/users/bhashemi/Projects/GIT/DY-GAN/delphes/total_Zmumu_13TeV_PU20_v2.npa', 'dropout_discriminator': False, 'terminate_early': True, 'do_batch_normalization_disc': False, 'scaler_type': 'jetiso', 'nepochs_dump_plots': 500, 'batch_size': 512, 'do_concatenate_disc': False, 'do_noisy_labels': False, 'do_soft_labels': False, 'depth_gen': 0, 'nepochs_dump_pred_metrics': 100, 'verbose': True, 'noise_size': 17, 'do_batch_normalization_gen': False, 'output_size': 17, 'frac_true': -1, 'loss_mll_weight': 0.0001, 'nepochs_max': 100001, 'beefy_discriminator': True, 'width_gen': 0, 'depth_disc': 0, 'do_tanh_gen': False, 'do_concatenate_gen': False, 'use_mll_loss': True, 'beefy_generator': True, 'nepochs_decay_noisy_labels': 1000, 'optimizer_gen': 'adadelta', 'noise_type': 3}
Scaling jet pts
Scaling lep isos
_______________

## Train

In [11]:
for i in range(1,20):
    params["tag"] = "jetisoscale_mllwidth_flatNegNoise_{}".format(i)
    params["verbose"] = False
    gan = GAN(**params)
    gan.train()
# gan.train()

Scaling jet pts
Scaling lep isos
Discriminator params: 161537
Generator params: 340497
scaling lepton isolations
scaling jet pts


/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


100 [D loss: 0.543494999409, acc.: 0.00%] [G loss: 1.34424138069] [mll=-1.000+--1.000] [ks=999.000]
KS score improved from 999.00 to 15.41, saving models to progress/jetisoscale_mllwidth_flatNegNoise_1/gen_100.weights
200 [D loss: 0.193427175283, acc.: 0.00%] [G loss: 4.62835168839] [mll=42.365+-4.936] [ks=15.409]
KS score improved from 15.41 to 14.96, saving models to progress/jetisoscale_mllwidth_flatNegNoise_1/gen_200.weights
300 [D loss: 0.177920266986, acc.: 0.00%] [G loss: 5.26860952377] [mll=50.267+-5.109] [ks=14.962]]
KS score improved from 14.96 to 14.31, saving models to progress/jetisoscale_mllwidth_flatNegNoise_1/gen_300.weights
400 [D loss: 0.127145722508, acc.: 0.00%] [G loss: 5.10356140137] [mll=37.862+-3.507] [ks=14.312]]
500 [D loss: 0.0876420065761, acc.: 0.00%] [G loss: 4.73637056351] [mll=69.735+-6.224] [ks=14.440]
10000/10000 [==============================] - 0s 46us/step


/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:838: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:780: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:781: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


KS score improved from 14.31 to 13.31, saving models to progress/jetisoscale_mllwidth_flatNegNoise_1/gen_500.weights
600 [D loss: 0.16150431335, acc.: 0.00%] [G loss: 6.39612531662] [mll=74.326+-6.391] [ks=13.314]]]
700 [D loss: 0.129242509604, acc.: 0.00%] [G loss: 5.88102388382] [mll=52.119+-3.219] [ks=14.043]]
800 [D loss: 0.495268255472, acc.: 50.00%] [G loss: 5.82737159729] [mll=48.116+-3.303] [ks=13.859]


...

79900 [D loss: 0.703518033028, acc.: 0.00%] [G loss: 0.70572668314] [mll=89.726+-4.628] [ks=7.800]]
80000 [D loss: 0.706376552582, acc.: 0.00%] [G loss: 0.705198705196] [mll=89.488+-4.862] [ks=8.403]
10000/10000 [==============================] - 0s 44us/step
80100 [D loss: 0.701454401016, acc.: 0.00%] [G loss: 0.704150795937] [mll=88.672+-4.115] [ks=7.824]
80200 [D loss: 0.710980296135, acc.: 0.00%] [G loss: 0.705721437931] [mll=88.625+-4.364] [ks=8.310]
80300 [D loss: 0.704851031303, acc.: 0.00%] [G loss: 0.705328822136] [mll=88.935+-4.266] [ks=7.992]
80400 [D loss: 0.709968626499, acc.: 0.00%] [G loss: 0.71222025156] [mll=90.077+-4.589] [ks=8.351]]
80500 [D loss: 0.706837892532, acc.: 0.00%] [G loss: 0.703678190708] [mll=86.308+-7.704] [ks=8.360]
10000/10000 [==============================] - 0s 49us/step
80600 [D loss: 0.70820248127, acc.: 0.00%] [G loss: 0.704286515713] [mll=89.847+-4.434] [ks=7.998]]
80700 [D loss: 0.710713267326, acc.: 0.00%] [G loss: 0.709072053432] [mll=89.888

87100 [D loss: 0.733048498631, acc.: 0.00%] [G loss: 0.7058801651] [mll=89.961+-4.484] [ks=7.772]]]
87200 [D loss: 0.709249377251, acc.: 0.00%] [G loss: 0.706842541695] [mll=89.225+-4.364] [ks=8.435]
87300 [D loss: 0.706650733948, acc.: 0.00%] [G loss: 0.705385446548] [mll=88.618+-4.400] [ks=8.102]
87400 [D loss: 0.708628177643, acc.: 0.00%] [G loss: 0.706606805325] [mll=89.361+-4.597] [ks=7.784]
87500 [D loss: 0.706632256508, acc.: 0.20%] [G loss: 0.703582763672] [mll=89.771+-4.195] [ks=7.861]
10000/10000 [==============================] - 0s 43us/step
87600 [D loss: 0.705144047737, acc.: 0.00%] [G loss: 0.71155923605] [mll=89.203+-5.326] [ks=7.883]]
87700 [D loss: 0.707520723343, acc.: 0.00%] [G loss: 0.706772327423] [mll=89.798+-4.457] [ks=7.874]
87800 [D loss: 0.72606742382, acc.: 0.00%] [G loss: 0.706362783909] [mll=89.603+-4.604] [ks=7.876]]
87900 [D loss: 0.707460641861, acc.: 0.00%] [G loss: 0.708558082581] [mll=90.528+-5.756] [ks=7.838]
88000 [D loss: 0.70593225956, acc.: 0.00

94400 [D loss: 0.707594633102, acc.: 0.00%] [G loss: 0.702052116394] [mll=88.774+-4.364] [ks=7.968]
94500 [D loss: 0.772324621677, acc.: 0.00%] [G loss: 0.708307802677] [mll=89.385+-4.766] [ks=7.581]
10000/10000 [==============================] - 0s 45us/step
94600 [D loss: 0.709205508232, acc.: 0.00%] [G loss: 0.707582890987] [mll=89.904+-3.899] [ks=7.803]
94700 [D loss: 0.708213090897, acc.: 0.00%] [G loss: 0.707558274269] [mll=89.332+-4.708] [ks=7.554]
94800 [D loss: 0.709797620773, acc.: 0.00%] [G loss: 0.713301599026] [mll=89.252+-4.024] [ks=7.680]
94900 [D loss: 0.703903198242, acc.: 0.00%] [G loss: 0.704571187496] [mll=89.721+-4.335] [ks=7.459]
95000 [D loss: 0.705792546272, acc.: 0.00%] [G loss: 0.722206652164] [mll=89.511+-4.524] [ks=7.753]
10000/10000 [==============================] - 0s 36us/step
95100 [D loss: 0.703872025013, acc.: 0.00%] [G loss: 0.705655992031] [mll=89.294+-4.375] [ks=8.012]
95200 [D loss: 0.702827692032, acc.: 0.00%] [G loss: 0.704945802689] [mll=89.487

1100 [D loss: 0.339876830578, acc.: 0.00%] [G loss: 5.37839078903] [mll=57.689+-5.178] [ks=14.783]]
1200 [D loss: 0.922900557518, acc.: 0.00%] [G loss: 2.95878458023] [mll=52.311+-4.667] [ks=14.376]]
1300 [D loss: 0.162435591221, acc.: 0.00%] [G loss: 5.69382572174] [mll=21.028+-2.106] [ks=15.375]]
1400 [D loss: 0.0481594949961, acc.: 0.00%] [G loss: 5.21388244629] [mll=63.929+-5.673] [ks=14.404]
KS score improved from 13.40 to 12.99, saving models to progress/jetisoscale_mllwidth_flatNegNoise_6/gen_1400.weights
1500 [D loss: 0.0526015982032, acc.: 0.00%] [G loss: 4.50280237198] [mll=83.351+-8.890] [ks=12.993]
10000/10000 [==============================] - 0s 46us/step
KS score improved from 12.99 to 12.04, saving models to progress/jetisoscale_mllwidth_flatNegNoise_6/gen_1500.weights
1600 [D loss: 0.0605621337891, acc.: 0.00%] [G loss: 3.95328307152] [mll=90.752+-8.024] [ks=12.037]
1700 [D loss: 0.203616887331, acc.: 0.00%] [G loss: 6.18505907059] [mll=78.119+-6.919] [ks=14.715]]
1800

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
preds = gan.predict(150000,1.0)
make_plots(preds,gan.data[:N])

## Plot metrics vs epoch

In [ ]:
# gan.tag = "v5_default_scan_0"
gan.load_last_checkpoint()
print "Loaded"

In [ ]:
def smooth(x,window=31,npoly=2):
    from scipy.signal import savgol_filter
    return savgol_filter(x,window,npoly)

fig, (ax1, ax2, ax3,ax4) = plt.subplots(1,4,figsize=(15,3))
# plot losses
ax1.plot(gan.d_epochinfo["epoch"],gan.d_epochinfo["d_loss"],color="C0",lw=0.5)
ax1.plot(gan.d_epochinfo["epoch"],gan.d_epochinfo["g_loss"],color="C1",lw=0.5)
ax1.plot(gan.d_epochinfo["epoch"],smooth(gan.d_epochinfo["d_loss"]),label="d loss (smoothed)",color="C0")
ax1.plot(gan.d_epochinfo["epoch"],smooth(gan.d_epochinfo["g_loss"]),label="g loss (smoothed)",color="C1")
ax1.legend()
# plot masses
ax2.plot(gan.d_epochinfo["epoch"],gan.d_epochinfo["mass_sig"],color="C0",lw=0.5)
ax2.plot(gan.d_epochinfo["epoch"],gan.d_epochinfo["mass_mu"],color="C1",lw=0.5)
ax2.plot(gan.d_epochinfo["epoch"],smooth(gan.d_epochinfo["mass_sig"]),label="Z width (smoothed)",color="C0")
ax2.plot(gan.d_epochinfo["epoch"],smooth(gan.d_epochinfo["mass_mu"]),label="Z mass (smoothed)",color="C1")
ax2.legend()
# plot ks metrics
ax3.plot(gan.d_epochinfo["epoch"],gan.d_epochinfo["ks"], lw=0.5,color="C0")
ax3.plot(gan.d_epochinfo["epoch"],smooth(gan.d_epochinfo["ks"]), label="KS metric (smooth)",color="C0")
ax3.legend();
# plot times
ax4.plot(gan.d_epochinfo["epoch"],1./60*(np.array(gan.d_epochinfo["time"])-gan.d_epochinfo["time"][0]), label="times [min]")
ax4.legend()

## Plot predictions and real events
Get the noise from the gan object, feed it into the generator, then also take true events (`data`)

In [ ]:
N = 15000
# frac = 0.0001 # 2 events
# frac = 0.00005 # 1 event
frac = 1.0
_, noise = gan.get_noise(N,max_true_samples_frac=frac)
# print preds.shape
preds = gan.generator.predict(noise,verbose=1)
make_plots(preds,gan.data[:N])

## Recarray
The real data (`gan.data`) is a recarray, so column indexing happens with `gan.data["lep1_px"]`.
The predictions from the generator are just normal matrices, so you have to do yucky things like `preds[:,7]`
and keep track of what that means. Use a helper function to get a `np.view` into the predictions so that
they columns can be accessed in the same way.

In [ ]:
def Z_pt(data,haspy2=False):
    pxsum = data["lep1_px"]+data["lep2_px"]
    if haspy2: pysum = data["lep1_py"]+data["lep2_py"]
    else: pysum = data["lep1_py"]
    return np.hypot(pxsum,pysum)

def met_pt(data):
    if "met" in data.dtype.names:
        return data["met"]
    else:
        return np.hypot(data["metx"],data["mety"])
    
def ht(data):
    return data["jet_pt1"]*(data["jet_pt1"] > 10) + \
           data["jet_pt2"]*(data["jet_pt2"] > 10) + \
           data["jet_pt3"]*(data["jet_pt3"] > 10) + \
           data["jet_pt4"]*(data["jet_pt4"] > 10) + \
           data["jet_pt5"]*(data["jet_pt5"] > 10)

def nvtx(data):
    if "mll" in data.dtype.names:
        return data.nvtxs
    else:
        return np.rint(data["nvtxs"])

In [ ]:
# plot Z_pT for predicted and real events
predsrec = get_recview(preds)
fig,ax = plt.subplots()
ax.hist(Z_pt(predsrec,haspy2=False),label="Z $p_{T}$ fake",bins=np.linspace(0,100,50),histtype="step",density=True,lw=2)
ax.hist(Z_pt(gan.data[:10000],haspy2=False), label="Z $p_{T}$ real",bins=np.linspace(0,100,50),histtype="step",density=True,lw=2)
ax.legend()
ax.set_title("Z-boson $p_T$ for real and fake events");

In [ ]:
fig,ax = plt.subplots()
lowpu_real_met  = met_pt(gan.data)[nvtx(gan.data) < 10]
highpu_real_met = met_pt(gan.data)[nvtx(gan.data) > 25]
lowpu_fake_met  = met_pt(predsrec)[nvtx(predsrec) < 10]
highpu_fake_met = met_pt(predsrec)[nvtx(predsrec) > 25]
_ = ax.hist(lowpu_real_met,label="low PU, real",bins=np.linspace(0,100,50),histtype="step",density=True,lw=1.5)
_ = ax.hist(lowpu_fake_met,label="low PU, fake",bins=np.linspace(0,100,50),histtype="step",density=True,lw=1.5)
_ = ax.hist(highpu_real_met,label="high PU, real",bins=np.linspace(0,100,50),histtype="step",density=True,lw=1.5)
_ = ax.hist(highpu_fake_met,label="high PU, fake",bins=np.linspace(0,100,50),histtype="step",density=True,lw=1.5)
_ = ax.legend()
_ = ax.set_title("MET for low (nvtx<10) and high (nvtx>25) events")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(9,4), sharex=True,sharey=True)

bins = [np.linspace(15,75,15),np.linspace(15,75,15)]

ptj1 = gan.data["jet_pt1"]
zpt = Z_pt(gan.data)
good = (zpt > 15.) & (ptj1 > 15.)
_,_,_,im = ax1.hist2d(ptj1[good],zpt[good],label="real",bins=bins,normed=True) #, norm=LogNorm())
fig.colorbar(im,ax=ax1, format="%.0e")

ptj1 = predsrec["jet_pt1"]
zpt = Z_pt(predsrec)
good = (zpt > 15.) & (ptj1 > 15.)
_,_,_,im = ax2.hist2d(ptj1[good],zpt[good],label="real",bins=bins,normed=True)
fig.colorbar(im,ax=ax2, format='%.0e')

ax1.set_title("Real")
ax2.set_title("Fake")
ax1.set_ylabel("Z $p_{T}$")
ax1.set_xlabel("jet 1 $p_{T}$")
ax2.set_xlabel("jet 1 $p_{T}$")


fig.set_tight_layout(True)


In [ ]:
predmll = Minv(preds)
predmll = predmll[np.isfinite(predmll)]
bins = np.linspace(60,120,120)
fig, (ax1, ax2) = plt.subplots(2,1, sharex=True,gridspec_kw={'height_ratios':[9, 2]})
hreal = ax1.hist(gan.data["mll"],bins=bins, label="real $m_{ll}$",histtype="step",density=True,lw=2)
hfake = ax1.hist(predmll,bins=bins, label="fake $m_{ll}$",histtype="step",density=True,lw=2)
ratio = hfake[0]/hreal[0]
ax2.plot(bins[:-1],ratio,marker="o",markersize=3,linewidth=1.5,linestyle="")
ax2.set_ylim([0.5,1.5])
ax2.set_ylabel("fake/real")
_ = ax1.legend()
realmll = gan.data["mll"]
print "real:",realmll.mean(), realmll.std()
print "fake:",predmll.mean(), predmll.std()

In [ ]:
jetmat = np.c_[predsrec["jet_pt1"],predsrec["jet_pt2"],predsrec["jet_pt3"],predsrec["jet_pt4"],predsrec["jet_pt5"]]
# njet counts just any jet with pt>15
njets1 = (jetmat > 15.).sum(axis=-1)
# njet counts only jets with pt>15 starting from first jet if all previous jets pass threshold
# e.g., if jetpt1>15, jetpt2<15, jetpt3>15, then njets==1 in second case, but 2 in first case
njets2 = (jetmat > 15.).argmin(axis=-1)
# turns out the two are very similar


fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(12,3), sharey=False)
for numer,denom,ax in [
    ["jet_pt2","jet_pt1",ax1],
    ["jet_pt3","jet_pt2",ax2],
    ["jet_pt4","jet_pt3",ax3],
]:
    jetratio_real = (gan.data[:N][numer]/gan.data[:N][denom])[(gan.data[:N][denom] > 15.) & (gan.data[:N][numer] > 15.)]
    jetratio_fake = (predsrec[numer]/predsrec[denom])[(predsrec[denom] > 15.) & (predsrec[numer] > 15.)]
    bins = np.linspace(0,4,60)
    ax.hist(jetratio_real,bins=bins,histtype="step",density=True,label="real",lw=1.5)
    ax.hist(jetratio_fake,bins=bins,histtype="step",density=True,label="fake",lw=1.5)
    ax.legend()
    ax.set_title("{} / {} ratio".format(numer,denom))
fig.set_tight_layout(True)

In [ ]:

import glob

picklenames = glob.glob("progress/v3_default_scan*/*pkl")
def get_epochks(fname):
    with open(fname) as fh:
        data = pickle.load(fh)
        return np.array(data["epoch"]),np.array(data["ks"])
    return [],[]
# print dirnames
# print get_epochks(picklenames[0])
fig,ax = plt.subplots()
for pn in picklenames:
    xs, ys = get_epochks(pn)
#     good = ys < 0.08
#     xs = xs[good]
#     ys = ys[good]
    print pn, xs[ys.argmin()], ys[ys.argmin()]
#     ax.plot(xs,ys, label=pn)
# ax.legend()